# Create Desktop App in Google Cloud Console and Authorize 

I used my school's email. In first run you need to autorize manually on browser. 
If you can't access it please let me know becaue school email is using mobile verification.

In [1218]:
# Jupyter Notebook is used.

In [1219]:
'Full authorization is given from Google Cloud Platform by following the method below'
#Create project-> API's and Services-> Libraries Allow API's-> Credentials-> oAuth

'Full authorization is given from Google Cloud Platform by following the method below'

In [1220]:
#necessary libraries are included for authorization and the other functions
from __future__ import print_function
import os.path
'Google Drive API and Sheets API'
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from apiclient.discovery import build
from apiclient.http import MediaFileUpload
import pandas as pd

In [1221]:
import win32com.client as win32

#### Slack API Libraries

In [1222]:
import slack
import os
from pathlib import Path
from dotenv import load_dotenv

#### Slack API Connection

In [1223]:
#environment file and bot token connection for slack bot
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)
#initialization slack bot
client = slack.WebClient(token=os.environ['SLACK_TOKEN'])

### Dataset.xlsx Preview

In [1224]:
#created a data frame to see what we have
df = pd.read_excel("Dataset.xlsx")
df.head()

,Campaign name,Total Impression,Total Clicks,CTR (%),CPC,Total App Install
0,A,106622,879,0.82,0.710692,626
1,B,95687,805,0.84,0.550902,506
2,C,12157,102,0.84,0.596041,57
3,D,6006,64,1.07,0.741919,47
4,E,67782,468,0.69,0.522319,274


In [1225]:


    # If modifying these scopes, delete the file token.json.
    SCOPES = ['https://www.googleapis.com/auth/drive']


    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'client_secrets.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    service = build('drive', 'v3', credentials=creds)

    # Call the Drive v3 API
    results = service.files().list(
        pageSize=10, fields="nextPageToken, files(id, name)").execute()
    items = results.get('files', [])

    if not items:
        print('No files found.')
    else:
        print('Files:')
        #sending notification message to "campaign" channel on Slack
        client.chat_postMessage(channel='#campaign',text="Connected to the Google Drive!")
        
        for item in items:
            print(u'{0} ({1})'.format(item['name'], item['id']))



Files:
O (1fAb5gT3DluAcPHiFx88_7pMXrodnQ_t5wRAai9gfVpw)
N (1hjA_CnFQu2rpbKCDZCP5gN-5K6-oe13YoQqnIcm_zG4)
M (1Bh7M1_5D4QmVFeIbti96G0KqHOgtLl2AZq36tKY0Ouo)
L (1-irmQe6-57uglMPyqd7awAPqoaUb6R7swlSzFWJOcmE)
K (1iYJGymXjfiVKtkXDhAXXB7GkdYV5XF-VhaQ9hUnK8uk)
H (1HL176zsfV_7Qq1lc1RP5FyCtUIDqSkJjULsqzvbJt5E)
G (145foG2HesdonoXQyzuGO2Yke8h_j9NKDXKwoaInr8M0)
F (1n4VsicrXXcurxJK6SI7TD3XZoxMiZn9xnMlJOXS51sA)
E (1oX87JYEVAH87fZFLnID2XhPG18fd3zL19nYuo0Iafcs)
D (10f6FjCYBkWimOw9vrVtu781SB2Id9-S-eSZXru2SxWI)


# 1-) Upload The Dataset To Google Drive

In [1226]:
def uploadFile(local_name,upload_name):
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text=upload_name+ " file is uploaded to Google Drive.")
    file_metadata = {'name': str(upload_name)}
    media = MediaFileUpload(str(local_name), mimetype='application/vnd.ms-excel')
    file = service.files().create(body=file_metadata,media_body=media,fields='id').execute()
    print ('File ID: %s' % file.get('id'))
    

In [1227]:
uploadFile("Dataset.xlsx","Dataset.xlsx")

File ID: 1_Z28B9w92YtATiQtsfdwBz55B-9QC8So


# 2-) Create A Folder That Contains The Datasets Spreadsheet In It Via API Methods.

###   2.1-) Create Folder

In [1228]:
def createFolder(name):
    name= str(name)
    file_metadata = {
        'name': name,
        'mimeType': 'application/vnd.google-apps.folder'
    }
    
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text= name+ " file is created on Google Drive.")
    
    file = service.files().create(body=file_metadata, fields='id').execute()
    print('Folder ID: %s' % file.get('id'))

In [1229]:
createFolder("Dataset")

Folder ID: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4


### 2.2-) Create Spreadsheet For Dataset

In [1230]:
#authorization, version and including the credentials for Spreadsheet
serviceSheets=build('sheets', 'v4', credentials=creds)

In [1231]:
def createSpreadSheet(title):
    title =str(title)
    serviceSheets=build('sheets', 'v4', credentials=creds)
    spreadsheet = {
    'properties': {
        'title': title,
        'locale': 'en_US',
        'timeZone':'Europe',
        'autoRecalc':'HOUR'
        }
    }
    
    spreadsheet = serviceSheets.spreadsheets().create(body=spreadsheet,fields='spreadsheetId').execute()
    ssheet_id='{0}'.format(spreadsheet.get('spreadsheetId'))
    
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text= title+ " file spreadsheet is created on google drive.")
    
    print("Spreadsheet is created")
    print('Spreadsheet ID: {0}'.format(spreadsheet.get('spreadsheetId')))
    return ssheet_id

In [1232]:
createSpreadSheet("Dataset")

Spreadsheet is created
Spreadsheet ID: 19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4


'19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4'

### 2.3-) Dataset To SpreadSheet

In [1233]:
#dataset is written to the spreadsheet

In [1234]:
def datasetToSpread(spread_id):
    mySpreadsheets = serviceSheets.spreadsheets().get(spreadsheetId=spread_id).execute()
    xlApp = win32.Dispatch('Excel.Application')
    wb = xlApp.Workbooks.Open(r"C:\Users\ASUS\Desktop\Keele Denemeler\Alictus\Dataset.xlsx")
    ws = wb.Worksheets("Sayfa1")
    rngData = ws.Range('A1').CurrentRegion()
    
    response = serviceSheets.spreadsheets().values().append(
    spreadsheetId=spread_id,
    valueInputOption='RAW',
    range='Sheet1!A1',
    body=dict(
        majorDimension='ROWS',
        values=rngData
    )
    ).execute()
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text= "Dataset is successfully written to the spreadsheet. Spreadsheet ID is"+spread_id)
    print("successfully written to:",spread_id)
    

In [1235]:
datasetToSpread("19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4")

successfully written to: 19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4


### 2.4-) Move File To Dataset Folder

In [1236]:
def moveFileToFolder(file_id,folder_id):
    file_id = str(file_id)
    folder_id = str(folder_id)

    # Retrieve the existing parents to remove
    file = service.files().get(fileId=file_id, fields='parents').execute()
    previous_parents = ",".join(file.get('parents'))

    # Move the file to the target folder
    file = service.files().update(
        fileId=file_id,
        addParents=folder_id,
        removeParents=previous_parents,
        fields='id, parents'
    ).execute()
    
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text= file_id+" File is transferred to the folder. Folder ID:"+folder_id)
    
    print("File transferred to the location:",folder_id)

In [1237]:
moveFileToFolder("19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4","1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4")

File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4


# 3-) Create Seperate Spreadsheets For Each Campaign

### Total Budget Calculation To G COLUMN

In [609]:
# need spreadsheet id and sheet_id

In [1238]:
def budgetCalculation(spread_id,sheet_id):
    
    # adding header to the G1
    request_body = {

        "majorDimension": "COLUMNS",
        "values": [
            ['Total Budget']
        ]
    }
    
    response = serviceSheets.spreadsheets().values().update(
    spreadsheetId=spread_id,
    valueInputOption='RAW',
    range='Sheet1!G1',
    body=request_body
    ).execute()
    
    request_body2 ={
      "requests": [
        {
          "repeatCell": {
            "range": {
              "sheetId": sheet_id,
              "startRowIndex": 1,
              "endRowIndex": 14,
              "startColumnIndex": 6,
              "endColumnIndex": 7
            },
            "cell": {
              "userEnteredValue": {
                  # =FLOOR(E2*C2) for integer values
                  "formulaValue": "=E2*C2"
              }
            },
            "fields": "userEnteredValue"
          }
        }
      ]
    }


    response = serviceSheets.spreadsheets().batchUpdate(
    spreadsheetId=spread_id,
    body=request_body2
    ).execute()
    
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text= "Total Budget is calculated for each campaign.")
    
    print("Spread id: {n} \n Sheet ID:{s}".format(n=spread_id,s=sheet_id))
    print("Calculation is completed and written to the G Column")

In [1239]:
budgetCalculation("19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4","0")

Spread id: 19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4 
 Sheet ID:0
Calculation is completed and written to the G Column


### 3.1-) Divide Each Row And Save Them As Different SpreadSheets 

In [1240]:
# Reaching To the Dataset SpreadSheet

In [1241]:
datasetSpread_id = "19UFExM6FdSVwcPZ3RhwYbWSxngMiUgWnaTgNLoVrsE4"

In [1242]:
response = serviceSheets.spreadsheets().values().get(
spreadsheetId=datasetSpread_id,
majorDimension='ROWS',
range='Sheet1!A1:G15'
).execute()

print("File Contains Below")
print(response)

File Contains Below
{'range': 'Sheet1!A1:G15', 'majorDimension': 'ROWS', 'values': [['Campaign name ', 'Total Impression', 'Total Clicks', 'CTR (%)', 'CPC', 'Total App Install', 'Total Budget'], ['A ', '106622', '879', '0.82', '0.7106916 ', '626', '624.6979164'], ['B', '95687', '805', '0.84', '0.5509020', '506', '443.47611'], ['C', '12157', '102', '0.84', '0.5960410', '57', '60.796182'], ['D', '6006', '64', ' 1.07 ', '0.7419186', '47', '47.4827904'], ['E', '67782', '468', ' 0.69 ', '0.5223186', '274', '244.4451048'], ['F', '4844', '49', ' 1.01', '0.6753181', '31', '33.0905869'], ['G', '7810', '54', ' 0.69', '0.5259081', '30', '28.3990374'], ['H', '9053', '69', ' 0.76', '0.5454837 ', '41', '37.6383753'], ['K', '13279', '101', ' 0.76', '0.4683506 ', '46', '47.3034106'], ['L', '18827', '123', ' 0.65', '0.4733046', '64', '58.2164658'], ['M', '16611', '108', ' 0.65', '0.4604180', '57', '49.725144'], ['N', '10246', '57', ' 0.56', '0.5342519 ', '27', '30.4523583'], ['O', '145467', '675', ' 0.

In [1243]:
#sending notification message to "campaign" channel on Slack
client.chat_postMessage(channel='#campaign',text= "Dataset values read.")

In [1244]:
#spread list is keeping the response values from spreadsheet file 
spread=[]

In [1245]:
spread=response['values']
#first row is header so we kept them inside header value
header=spread[0]

In [1246]:
header

['Campaign name ',
 'Total Impression',
 'Total Clicks',
 'CTR (%)',
 'CPC',
 'Total App Install',
 'Total Budget']

In [1247]:
#spreadIdlist is keeping the file Id's 

In [1248]:
spreadIdlist = []

### 3.2-) Creating Campaign Spreadsheets For Each Row and Moving Them To Dataset Folder

In [1249]:
for i in range(1,len(spread)):
    spread_name=spread[i][0]
    body = spread[i]
    title =spread_name
    spreadsheet = {
        'properties': {
            'title': title,
            'locale': 'en_US',
            'timeZone':'Europe',
            'autoRecalc':'HOUR'
        }
    }

    spreadsheet = serviceSheets.spreadsheets().create(body=spreadsheet,fields='spreadsheetId').execute()

    spreads_id=spreadsheet.get('spreadsheetId')
    request_body = {
        "majorDimension": "ROWS",
        "values": [
            header,body
          ]
        }

    response = serviceSheets.spreadsheets().values().append(
    spreadsheetId=spreads_id,
    valueInputOption='RAW',
    range='Sheet1!A1',
    body=request_body
    ).execute()

    spreadIdlist.append(spreads_id)
    
    #sending notification message to "campaign" channel on Slack
    client.chat_postMessage(channel='#campaign',text= "Spreadsheet is created for campaign "+ title +" Spreadsheet ID:"+spreads_id)
    
    print("Campaign written to Spreadsheet:",spreads_id)


Campaign written to Spreadsheet: 1IGInQc2HfTCTiE4FDcpz1R7AVShiG8ePHHhcs-MzZzg
Campaign written to Spreadsheet: 10ceRt1XSjlzpQfljQQkrz8T2wVMlORhV-4d5HJIa95M
Campaign written to Spreadsheet: 17dNaudWASQLC2N3TRX8MEohAOLSRmYUiu7q9Kf7U2qY
Campaign written to Spreadsheet: 1osZjVCvoNCNhzN2dcw5-j-DMcfdhmGWIZbURhXRJHiI
Campaign written to Spreadsheet: 1Kfadg3kDXuKJh8jJxuPb4vwyKF3Oa-BSSzR17dED8pI
Campaign written to Spreadsheet: 1QPR9_oagKnMAA5XI5a6sfkSvnpWzi03BTrU13-UdSJs
Campaign written to Spreadsheet: 1ak7yirJJRwtHN_NjX9W7mj2gU0zP6bmzLdj8QEb9DSo
Campaign written to Spreadsheet: 1RmerFfgzkjaeZiHFwmhKMwOHLPrP8CsB4EJBjxanWFM
Campaign written to Spreadsheet: 1GjUJAvfIndedNNl1VrC-4adTzAI_iKbkFzqt6KwQgDU
Campaign written to Spreadsheet: 1jGqJTV-lUHBtwvviuEbRYsxgrcnDSZ8gQ7CpoHb2M9A
Campaign written to Spreadsheet: 1l2myzu6RWGo4uvi2WIGsrk9avswrRoHZmO2o35j1uZM
Campaign written to Spreadsheet: 1rjdMBTycNUV45OYRs1TB02DiOJ0zr09cMQqv4x5eoQU
Campaign written to Spreadsheet: 1xqtvOgtIJjc8YvA2CYMYEgW4IdZg8U

### 3.3) Transfer All Campaign Spreadsheets To Dataset Folder

In [1250]:
for i in range(0,len(spreadIdlist)):
    moveFileToFolder(spreadIdlist[i],"1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4")
    

File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4
File transferred to the location: 1lNeDYhwqA4T-9wnaApkj0VoQROQkPDc4


# 4-) Slack API-Bot (Slack is included in the beginning.)

## This is just a reminder for the step 4 is finished.

Notification messages are added to each function.

In [772]:
#sending message with the method below

# 5-) Slack API Calculate-Budget Slash Command

In [1251]:
#to get and post request, we need flask library.

In [1252]:
from flask import Flask, request, Response
from slackeventsapi import SlackEventAdapter

In [1253]:
app = Flask(__name__)
slack_event_adapter= SlackEventAdapter(os.environ['SIGNING_SECRET'],'/slack/events',app)

In [1254]:
client=slack.WebClient(token=os.environ['SLACK_TOKEN'])


In [1255]:
#Campaign dictionary to hold the total budget values.

In [1256]:
dicti={}

In [1257]:
for i in range(1,len(spread)):
    index=spread[i][0].strip()
    dicti[index]={spread[i][6]}

In [1212]:
#Slack API sends the campaign name and SearchResult returns the budget 

In [1258]:
def searchResult(text):
    text = str(text)
    result ="Campaign "+ text +" Total Budget is " + str(dicti[text])
    return result
    

In [1259]:
#Slack API Slash Command 

In [1260]:
@app.route('/calculate-budget', methods=['GET','POST'])
def calculate_budget():
    #get data, userId,channelId and Text.
    data = request.form
    user_id= data.get('user_id')
    channel_id =data.get('channel_id')
    text = data.get('text')
    
    #transform it to uppercase and send to SearchResult function to get the Total Budget.
    send_text=text.upper()
    #send value keeps the text client will see.
    send=searchResult(send_text)
    client.chat_postMessage(channel=channel_id,text=send)
    return Response(), 200

In [1261]:
#initialize the flask service
#then ngrok.exe needs to be started.
# enter 'ngrok http 5000' to start it on its prompt

In [1262]:
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/Oct/2021 03:30:41] "POST /calculate-budget HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 03:30:50] "POST /calculate-budget HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 03:30:56] "POST /calculate-budget HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 03:31:07] "POST /calculate-budget HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2021 03:31:15] "POST /calculate-budget HTTP/1.1" 200 -
